### Select lakes in hma from global dataset

In [2]:
import geopandas as gpd

In [13]:
path_hma_basins_l4 = 'data/hma-extent/basins/hma_Lehner2013_levels/hma_hybas_as_lev04_v1c.gpkg' 
path_lakes_global = 'data/water-extent/lakes/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp' 


In [14]:
## Read data
global_lakes_gdf = gpd.read_file(path_lakes_global)
hma_basin_gdf = gpd.read_file(path_hma_basins_l4)
print(global_lakes_gdf.shape)
global_lakes_gdf.head(2)


(1427688, 22)


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,1,Caspian Sea,Russia,Europe,SWBD,1,0,377001.91,15829.37,7.27,...,1,200.5,8110.642,107883.0,-29,-1.0,1404108.0,47.717708,45.591934,"POLYGON ((49.96181 37.43847, 49.96457 37.44022..."
1,2,Great Bear,Canada,North America,CanVec,1,0,30450.64,5331.72,8.62,...,1,72.2,535.187,47577.7,145,-1.0,147665.4,-123.505546,65.138384,"POLYGON ((-119.78782 67.03574, -119.78637 67.0..."


In [ ]:
hma_basin_lake_gdf = global_lakes_gdf[global_lakes_gdf.intersects(hma_basin_gdf.union_all())]
#### Write out
# hma_basin_lake_gdf.to_file(filename='data/water-extent/lakes/HydroLakes_v10_hma_basin.gpkg', driver='GPKG')
print(hma_basin_lake_gdf.shape)
hma_basin_lake_gdf.head(2)


(12807, 22)


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
11,12,Balkhash,Kazakhstan,Asia,SWBD,1,0,16717.89,3092.20,6.75,...,1,6.7,470.083,2745.3,338,-1.0,404800.5,74.602083,46.385417,"POLYGON ((74.98387 46.83114, 74.98618 46.82939..."
129,130,Small Aral Sea,Kazakhstan,Asia,SWBD,1,0,2964.43,489.88,2.54,...,1,26.8,362.486,2536.7,39,-1.0,325885.8,60.952083,46.335417,"POLYGON ((60.59829 46.77318, 60.6109 46.77293,..."


#### statistics of the lakes in the hma basins.

In [58]:
joined_gdf = gpd.sjoin(left_df = hma_basin_gdf,
                        right_df = hma_basin_lake_gdf,
                        how='inner',
                        predicate='intersects')


In [91]:
stats = joined_gdf.groupby('HYBAS_ID').agg(
    lake_count=('Hylak_id', 'size'),     
    lake_area=('Lake_area', 'sum')
    ).reset_index()
stats.head()


,HYBAS_ID,lake_count,lake_area
0,4040023060,202,562.65
1,4040023810,426,1034.32
2,4040033430,532,1297.73
3,4040050210,13,7.88
4,4040050240,552,8289.21


In [ ]:
hma_basin_lake_stats = hma_basin_gdf.merge(stats, on='HYBAS_ID', how='left')
hma_basin_lake_stats = hma_basin_lake_stats[['HYBAS_ID', 'lake_count', 'lake_area', 'geometry']]
hma_basin_lake_stats.head()
# hma_basin_lake_stats.to_file(filename='data/water-extent/lakes/HydroLakes_v10_hma_basin_stats.gpkg', driver='GPKG') 
